In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark

In [2]:
from pyspark import SparkContext
sc=SparkContext()

In [3]:
from pyspark.sql import SQLContext
sqlContext=SQLContext(sc)

In [103]:
from pyspark.sql.functions import col,isnan,when,count,split,length,trim,sum,avg,desc,round
from pyspark.sql.types import (
    StringType, BooleanType, IntegerType, FloatType, DateType
)


In [135]:
bdoriginal=sqlContext.read.format("com.databricks.spark.csv").option("header","true").load("complaints.csv", inferSchema=True)
sqlContext.registerDataFrameAsTable(bdoriginal,"bd")

In [136]:
bdoriginal.count()

4876519

In [137]:
bdoriginal.show()

+-------------+--------------------+----------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+--------------------------+-------------+--------------------+----------------------------+----------------+------------------+------------+
|Date received|             Product|     Sub-product|               Issue|           Sub-issue|Consumer complaint narrative|Company public response|             Company|State|ZIP code|         Tags|Consumer consent provided?|Submitted via|Date sent to company|Company response to consumer|Timely response?|Consumer disputed?|Complaint ID|
+-------------+--------------------+----------------+--------------------+--------------------+----------------------------+-----------------------+--------------------+-----+--------+-------------+--------------------------+-------------+--------------------+----------------------------+----------------+----------------

In [147]:

bd = bdoriginal.withColumn('year', split(bdoriginal['Date received'], '-').getItem(0)) \
       .withColumn('month', split(bdoriginal['Date received'], '-').getItem(1)) \
       .withColumn('day', split(bdoriginal['Date received'], '-').getItem(2))
bd = bd.filter("day is not null and month is not null and year is not null")
bd = bd.filter("Product is not null")
bd = bd.filter(length(col("day")) <3)
bd = bd.filter(length(col("Product")) <20)
bd.show(truncate=False)


bd.count()


+-------------+---------------+--------------------------+-----------------------------------------------------------+--------------------------------------------------------------+----------------------------+-----------------------------------------------------------------------------------------------+--------------------------------------+-----+--------+-----------------------------+--------------------------+-------------+--------------------+----------------------------+----------------+------------------+------------+----+-----+---+
|Date received|Product        |Sub-product               |Issue                                                      |Sub-issue                                                     |Consumer complaint narrative|Company public response                                                                        |Company                               |State|ZIP code|Tags                         |Consumer consent provided?|Submitted via|Date sent to company|Co

1184424

In [78]:
def group_item(columna, cantidad):
    
    df = bd.select(columna)
    df = df.groupBy(columna).count()
    df = df.sort(col("count").desc())
    total = df.agg({'count': 'sum'}).collect()[0][0]
    df = df.withColumn("avg",round(col("count")/total,2))
    df = df.filter("count > " + cantidad)
    return df

In [148]:
df = group_item("Product","100")
df.show(df.count())

+----------------+------+----+
|         Product| count| avg|
+----------------+------+----+
| Debt collection|464901|0.39|
|        Mortgage|370360|0.31|
|Credit reporting|140429|0.12|
|     Credit card| 89190|0.08|
|    Student loan| 72929|0.06|
|   Consumer Loan| 31596|0.03|
|     Payday loan|  5543| 0.0|
| Money transfers|  5354| 0.0|
|    Prepaid card|  3819| 0.0|
+----------------+------+----+



In [149]:
df_select = bd.where("Product = 'Mortgage' OR Product = 'Debt collection'  OR Product = 'Credit reporting'  OR Product = 'Credit card' OR Product = 'Student loan' OR Product = 'Consumer Loan'")

In [151]:
df_select.count()

1169405

In [152]:
df_select = df_select.select("Product","Sub-product","Issue","Company","State","ZIP code","Submitted via","Consumer disputed?","year","month")
df_select.show()

+---------------+--------------------+--------------------+--------------------+-----+--------+-------------+------------------+----+-----+
|        Product|         Sub-product|               Issue|             Company|State|ZIP code|Submitted via|Consumer disputed?|year|month|
+---------------+--------------------+--------------------+--------------------+-----+--------+-------------+------------------+----+-----+
|       Mortgage|Conventional home...|Trouble during pa...| Selene Holdings LLC|   AZ|   85255|          Web|               N/A|2023|   01|
|Debt collection|    Credit card debt|False statements ...|Dimand Law Office...|   IL|   60652|          Web|               N/A|2023|   02|
|Debt collection|       I do not know|Attempts to colle...|Aldridge Pite Haa...|   GA|   30047|          Web|               N/A|2023|   02|
|       Mortgage|Conventional home...|Applying for a mo...|Rocket Mortgage, LLC|   NE|   68066|          Web|               N/A|2023|   02|
|Debt collection|   

In [153]:
df_Mortgage = df_select.filter("Product = 'Mortgage'")
df_Mortgage.show()

+--------+--------------------+--------------------+--------------------+-----+--------+-------------+------------------+----+-----+
| Product|         Sub-product|               Issue|             Company|State|ZIP code|Submitted via|Consumer disputed?|year|month|
+--------+--------------------+--------------------+--------------------+-----+--------+-------------+------------------+----+-----+
|Mortgage|Conventional home...|Trouble during pa...| Selene Holdings LLC|   AZ|   85255|          Web|               N/A|2023|   01|
|Mortgage|Conventional home...|Applying for a mo...|Rocket Mortgage, LLC|   NE|   68066|          Web|               N/A|2023|   02|
|Mortgage|Conventional home...|Struggling to pay...|WELLS FARGO & COM...|   TX|   78258|          Web|               N/A|2023|   01|
|Mortgage|Conventional home...|Closing on a mort...|CARRINGTON MORTGA...|   FL|   33579|          Web|               N/A|2023|   01|
|Mortgage|Conventional home...|Applying for a mo...|WELLS FARGO & COM

In [155]:
df = group_item("Sub-product","100")
df.show(df.count())


+--------------------+------+----+
|         Sub-product| count| avg|
+--------------------+------+----+
|                null|235188| 0.2|
|       I do not know|102188|0.09|
|          Other debt| 88017|0.07|
|Conventional home...| 87542|0.07|
|      Other mortgage| 86633|0.07|
|    Credit card debt| 75096|0.06|
|Conventional fixe...| 70613|0.06|
|        Medical debt| 49418|0.04|
|        FHA mortgage| 46170|0.04|
|Other (i.e. phone...| 44534|0.04|
|Federal student l...| 37163|0.03|
|         Credit card| 28693|0.02|
|Conventional adju...| 25380|0.02|
|Non-federal stude...| 25163|0.02|
|             Medical| 21184|0.02|
|        Vehicle loan| 17779|0.02|
|Other type of mor...| 14916|0.01|
|         VA mortgage| 14272|0.01|
|Private student loan| 13483|0.01|
|           Auto debt| 12596|0.01|
|Home equity loan ...| 11623|0.01|
|    Installment loan|  8696|0.01|
|    Payday loan debt|  8550|0.01|
|Home equity loan ...|  8346|0.01|
|         Payday loan|  7558|0.01|
|       Mortgage deb

In [156]:
df = group_item("Issue","100")
df.show(df.count())

+--------------------+------+----+
|               Issue| count| avg|
+--------------------+------+----+
|Attempts to colle...|159583|0.13|
|Loan modification...|112307|0.09|
|Incorrect informa...|102684|0.09|
|Loan servicing, p...| 77332|0.07|
|Written notificat...| 71664|0.06|
|Trouble during pa...| 66596|0.06|
|Cont'd attempts c...| 60665|0.05|
|Communication tac...| 51215|0.04|
|Struggling to pay...| 40198|0.03|
|False statements ...| 37986|0.03|
|Disclosure verifi...| 30790|0.03|
|Took or threatene...| 27657|0.02|
|Dealing with your...| 27227|0.02|
|Applying for a mo...| 19428|0.02|
|Dealing with my l...| 17630|0.01|
|Application, orig...| 17229|0.01|
|Credit reporting ...| 16883|0.01|
|Managing the loan...| 15278|0.01|
|    Billing disputes| 15136|0.01|
|               Other| 14779|0.01|
|Closing on a mort...| 12218|0.01|
|Unable to get cre...| 10859|0.01|
|Improper contact ...| 10068|0.01|
|Problems when you...|  9383|0.01|
|Settlement proces...|  8940|0.01|
|Taking/threatenin..

In [157]:
df = group_item("Consumer disputed?","100")
df.show(df.count())

+--------------------+------+----+
|  Consumer disputed?| count| avg|
+--------------------+------+----+
|                  No|508876|0.43|
|                 N/A|395057|0.33|
|                null|139863|0.12|
|                 Yes|122487| 0.1|
|Closed with expla...|  2924| 0.0|
|                 Web|  2038| 0.0|
|    Consent provided|  1667| 0.0|
| or delete the it...|   804| 0.0|
|Closed with non-m...|   342| 0.0|
|Closed with monet...|   178| 0.0|
|       Servicemember|   136| 0.0|
|                  CA|   123| 0.0|
|Company has respo...|   115| 0.0|
|      Older American|   101| 0.0|
+--------------------+------+----+



In [158]:
df = group_item("year","100")
df.show(df.count())

+----+------+----+
|year| count| avg|
+----+------+----+
|2016|169142|0.14|
|2015|151012|0.13|
|2014|138251|0.12|
|2017|124331| 0.1|
|2021|101007|0.09|
|2013| 94827|0.08|
|2022| 90721|0.08|
|2018| 84496|0.07|
|2020| 82972|0.07|
|2019| 76294|0.06|
|2012| 60160|0.05|
|2023|  8390|0.01|
|2011|  2536| 0.0|
|    |   112| 0.0|
+----+------+----+



In [159]:
df = group_item("company","100")
df.show(df.count())

+--------------------+------+----+
|             company| count| avg|
+--------------------+------+----+
|                null|134515|0.11|
|BANK OF AMERICA, ...| 56802|0.05|
|       EQUIFAX, INC.| 51624|0.04|
|WELLS FARGO & COM...| 50701|0.04|
|Experian Informat...| 48881|0.04|
|TRANSUNION INTERM...| 43717|0.04|
|JPMORGAN CHASE & CO.| 36930|0.03|
|      CITIBANK, N.A.| 32409|0.03|
|Ocwen Financial C...| 28924|0.02|
|Navient Solutions...| 28836|0.02|
|CAPITAL ONE FINAN...| 22251|0.02|
|NATIONSTAR MORTGA...| 22189|0.02|
|PORTFOLIO RECOVER...| 16232|0.01|
|ENCORE CAPITAL GR...| 15647|0.01|
| SYNCHRONY FINANCIAL| 15055|0.01|
|Ditech Financial LLC| 12943|0.01|
|        U.S. BANCORP| 10607|0.01|
|SELECT PORTFOLIO ...| 10079|0.01|
|           AES/PHEAA|  9691|0.01|
|                 ERC|  8971|0.01|
|Resurgent Capital...|  8680|0.01|
|AMERICAN EXPRESS ...|  8318|0.01|
|       DISCOVER BANK|  7588|0.01|
|       PNC Bank N.A.|  6874|0.01|
|HSBC NORTH AMERIC...|  6809|0.01|
|TRANSWORLD SYSTEM..

In [160]:
df = group_item("state","100")
df.show(df.count())

+--------------------+------+----+
|               state| count| avg|
+--------------------+------+----+
|                null|147867|0.12|
|                  CA|133889|0.11|
|                  FL|100712|0.09|
|                  TX| 92540|0.08|
|                  NY| 64954|0.05|
|                  GA| 55547|0.05|
|                  NJ| 38028|0.03|
|                  IL| 36451|0.03|
|                  PA| 35915|0.03|
|                  VA| 30952|0.03|
|                  OH| 30820|0.03|
|                  NC| 29962|0.03|
|                  MD| 29962|0.03|
|                  MI| 25087|0.02|
|                  AZ| 21813|0.02|
|                  WA| 19372|0.02|
|                  MA| 18860|0.02|
|                  TN| 17263|0.01|
|                  SC| 16338|0.01|
|                  CO| 16204|0.01|
|                  MO| 14018|0.01|
|                  NV| 12913|0.01|
|                  LA| 12906|0.01|
|                  AL| 12396|0.01|
|                  IN| 11700|0.01|
|                  C

In [118]:
def unificar(df, nombre_archivo):
    df2 = df.repartition(1)
    return df2.write.option("header",True).csv(nombre_archivo)

In [128]:
df_consulta = df_select #.filter("product = 'Mortgage'")
df_consulta = df_consulta.groupBy("company").count()
df_consulta = df_consulta.filter("count > 100")
df_consulta.sort(col("count").desc()).show(df_consulta.count())

+--------------------+-----+
|             company|count|
+--------------------+-----+
|                null|48285|
|     Bank of America|42398|
|             Equifax|27606|
|Wells Fargo & Com...|27016|
|            Experian|26949|
|JPMorgan Chase & Co.|23328|
|TransUnion Interm...|21660|
|            Citibank|19674|
|               Ocwen|19285|
| Nationstar Mortgage|11859|
|         Capital One|11577|
|Ditech Financial LLC| 7533|
| Synchrony Financial| 7126|
|Encore Capital Group| 5375|
|        U.S. Bancorp| 5313|
|HSBC North Americ...| 4856|
|                Amex| 4473|
|Select Portfolio ...| 3834|
|       PNC Bank N.A.| 3688|
|            Discover| 3343|
|Portfolio Recover...| 2677|
|SunTrust Banks, Inc.| 2506|
|       Seterus, Inc.| 2431|
|        Barclays PLC| 2229|
|                 ERC| 2061|
|Specialized Loan ...| 1708|
|M&T Bank Corporation| 1536|
|       Flagstar Bank| 1532|
|Expert Global Sol...| 1514|
|TD Bank US Holdin...| 1500|
|Resurgent Capital...| 1466|
|CIT Bank Nati

In [129]:
print(unificar(df_consulta,"company_counts"))

None
